# cbow_predict

In [ ]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul # common folder의 layers.py 파일에서 MatMul class


# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]]) # you에 대응하는 희소벡터 ([[]] : 1*7 행렬로 이해하겠다는 의미.)
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]]) # goodbye에 대응하는 희소벡터

# 가중치 초기화
W_in = np.random.randn(7, 3) # 7*3 행렬을 만들겠다.
W_out = np.random.randn(3, 7) # 3*7 행렬을 만들겠다.

# 계층 생성
in_layer0 = MatMul(W_in) # MatMul층은 Affine층 중 특별한 것, 매개변수로 bias는 없고 weight만 있음.
in_layer1 = MatMul(W_in) # 입력 MatMul층 2개, Weight mat 동일...
out_layer = MatMul(W_out)

# 순전파
h0 = in_layer0.forward(c0) # c0 통과시키면 h0
h1 = in_layer1.forward(c1) # c1 통과시키면 h1
h = 0.5 * (h0 + h1) # 두개 평균
s = out_layer.forward(h) # activation func는 없음.
print(s)

# MatMul
/common/layers.py/

In [ ]:
from common.np import *  # import numpy as np
from common.config import GPU
from common.functions import softmax, cross_entropy_error


class MatMul:
    def __init__(self, W): # MatMul층은 Affine층 중 특별한 것, 매개변수로 bias는 없고 weight만 있음.
        self.params = [W] # Weight: []안에 들어가 있음. 다른 층들이랑 일관성 유지하기 위해.
        self.grads = [np.zeros_like(W)]
        self.x = None

    def forward(self, x): # 순전파
        W, = self.params # 위 초기화 함수에서의 인자 W. *** list,를 하면 []안에 있는 요소만 나옴.
        out = np.dot(x, W)
        self.x = x
        return out

    def backward(self, dout): # 역전파
        W, = self.params
        dx = np.dot(dout, W.T) # dout에 Weight mat Transpose해서 곱해줘라.
        dW = np.dot(self.x.T, dout) # x를 Transpose해서 곱해줘라.
        self.grads[0][...] = dW # []안에 있는 걸 dW로 바꿔라. ***[...]: deep copy 깊은 복사, shape 유지
        return dx

# simple_cbow
* 일반적인 내용

In [ ]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul, SoftmaxWithLoss


class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size): # vocab_size 어휘개수: 입력층 뉴런 개수, # 은닉층 뉴런 개수: 단어를 벡터로 표현할 때 벡터 차원
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f') # 표준편차 0.01, 'f': 32bit라는 소리
        W_out = 0.01 * np.random.randn(H, V).astype('f')

        # 계층 생성
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss() # cbow_predict.py에는 없던 softmax & cross_entropy 층

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer0, self.in_layer1, self.out_layer] # matmul의 인스턴스 3개, self_in_layer끼리는 동일 (weight이므로)
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target): # 문맥, # 타겟(예측시 정답)
        h0 = self.in_layer0.forward(contexts[:, 0])
        h1 = self.in_layer1.forward(contexts[:, 1])
        h = (h0 + h1) * 0.5
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss # 손실함수 값 리턴 46:00...

    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer.backward(ds)
        da *= 0.5
        self.in_layer1.backward(da)
        self.in_layer0.backward(da)
        return None


# train.py

In [ ]:
import sys
sys.path.append('..')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.trainer import Trainer
from common.optimizer import Adam
from simple_cbow import SimpleCBOW
from common.util import preprocess, create_contexts_target, convert_one_hot


window_size = 1
hidden_size = 5
batch_size = 3
max_epoch = 1000

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

vocab_size = len(word_to_id)
contexts, target = create_contexts_target(corpus, window_size)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

model = SimpleCBOW(vocab_size, hidden_size)
optimizer = Adam()
trainer = Trainer(model, optimizer)

trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

word_vecs = model.word_vecs # input mat
for word_id, word in id_to_word.items():
    print(word, word_vecs[word_id])


# util.py

# trainer.py

# skip_gram.py

In [ ]:
# layer_in : input weight
# h: hidden
# s: output
# cbow: label = target (양 옆의 단어로 중앙의 단어 예측)
# skip: label = context (중앙의 단어로 양 옆 단어 예측)

# grads update...

import simple_skip_gram import SimpleSkipGram